Adapted from:

[1] M. G. Berman et al., ‘The Perception of Naturalness Correlates with Low-Level Visual Features of Environmental Scenes’, PLOS ONE, vol. 9, no. 12, p. e114572, Dec. 2014, doi: 10.1371/journal.pone.0114572.

In [ ]:
import os
import json

import cv2
import numpy as np
from skimage import color, feature, filters, measure
import matplotlib.pyplot as plt

from utils import *

In [ ]:
folder_path = "./data/Most_extreme_pictures"

In [ ]:
def apply_green_mask(img, straight_mask, non_straight_mask, green_mask):
    total_pixels = img.size
    hsv_image = color.rgb2hsv(img)
    hue_mask = hsv_image[:, :, 0]

    if not np.any(green_mask):
        green_mask = (hsv_image[:, :, 0] > 0.1) & (hsv_image[:, :, 0] < 0.4)

    non_straight_with_green = non_straight_mask & green_mask
    non_straight_density_with_green = non_straight_with_green.sum() / total_pixels
    straight_density = straight_mask.sum() / total_pixels

    visualization = np.zeros((*straight_mask.shape, 3), dtype=np.uint8)
    visualization[straight_mask] = [255, 255, 255]  # White for straight edges
    visualization[non_straight_with_green] = [0, 255, 0]  # Green for non-straight with green

    plt.figure()
    im = plt.imshow(hue_mask, cmap='hsv')
    plt.title("Hue Mask")
    plt.axis('off')
    plt.colorbar(im)
    plt.show()    

    plt.figure()
    plt.imshow(green_mask, cmap='Greens')
    plt.title("Green Mask")
    plt.axis('off')
    plt.show()    

    plt.figure()
    plt.imshow(visualization)
    plt.title(f"Straight and Non-Straight with Green Edges, SED: {straight_density:.4f}, "
              f"NSD with Green: {non_straight_density_with_green:.4f}")
    plt.axis('off')
    plt.show()

    return non_straight_density_with_green


def detect_straight_edges(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY).astype(np.float64) / 255.0
    total_pixels = gray.size

    edges = feature.canny(gray, sigma=1.0)
    smoothed = filters.gaussian(gray, sigma=1)
    Gx, Gy = np.gradient(smoothed)
    theta = np.mod(np.degrees(np.arctan2(Gy, Gx)), 360)
    bins = np.linspace(0, 360, 9)
    dir_bins = np.digitize(theta, bins) - 1

    straight_mask = np.zeros_like(edges, dtype=bool)
    for d in range(8):
        mask = edges & (dir_bins == d)
        if np.count_nonzero(mask) < 5:
            continue
        labels = measure.label(mask, connectivity=2)
        for region in measure.regionprops(labels):
            coords = region.coords
            if coords.shape[0] < 5:
                continue
            y, x = coords[:, 0], coords[:, 1]
            cov = np.cov(x, y)
            eigvals = np.linalg.svd(cov, compute_uv=False)
            if eigvals[0] > 104 * eigvals[1]:
                straight_mask[y, x] = True

    non_straight_mask = edges & ~straight_mask

    straight_density = straight_mask.sum() / total_pixels
    non_straight_density = non_straight_mask.sum() / total_pixels

    return straight_density, non_straight_density, straight_mask, non_straight_mask


def fractal_dimension_boxcount(bw):
    rows, cols = bw.shape
    max_box = min(rows, cols)
    sizes = 2 ** np.arange(int(np.floor(np.log2(max_box))), 0, -1)
    counts = []
    for size in sizes:
        count = 0
        for r in range(0, rows, size):
            for c in range(0, cols, size):
                sub = bw[r:r+size, c:c+size]
                if np.any(sub):
                    count += 1
        counts.append(count)
    log_counts = np.log(counts)
    log_sizes = np.log(1.0 / sizes)
    coeffs = np.polyfit(log_sizes, log_counts, 1)
    return coeffs[0]


def process_image(file_path):
    file_name = os.path.basename(file_path)

    img_bgr = cv2.imread(file_path)
    img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    red = img[:, :, 0]
    green = img[:, :, 1]
    blue = img[:, :, 2]
    
    hsv = color.rgb2hsv(img)
    H = hsv[:, :, 0]

    green_mask = (green > red) & (green > blue)
    redcyan_mask = (red > green) & (red > blue)
    gv_hue = H[green_mask].mean() if np.any(green_mask) else np.nan
    rc_hue = H[redcyan_mask].mean() if np.any(redcyan_mask) else np.nan
    hsv_hue = H.mean()
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    brightness = gray.mean()
    sd_brightness = gray.std()
    S = hsv[:, :, 1]
    saturation = S.mean()
    sd_saturation = S.std()

    sed, nsed, straight_mask, non_straight_mask = detect_straight_edges(img)
    nsed_green = apply_green_mask(img, straight_mask, non_straight_mask, green_mask)

    entropy_val = measure.shannon_entropy(gray, base=2)
    _, bw = cv2.threshold(gray, 0, 1, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    fractal_d = fractal_dimension_boxcount(bw.astype(bool))

    result = {
        'ImageName': file_name,
        'gvHue': gv_hue,
        'rcHue': rc_hue,
        'HSVHue': hsv_hue,
        'Brightness': brightness,
        'sdBrightness': sd_brightness,
        'Saturation': saturation,
        'sdSaturation': sd_saturation,
        'StraightEdgeDensity': sed,
        'NonStraightEdgeDensity': nsed,
        'NonStraightEdgeDensityWithGreen': nsed_green,
        'Entropy': entropy_val,
        'FractalD': fractal_d
    }

    rounded_result = {key: round(value, 4) if isinstance(value, float) else value for key, value in result.items()}
    print(json.dumps(rounded_result, indent=4))

    plt.figure()
    plt.imshow(img)
    plt.title("Original Image")
    plt.show()

    plt.figure()
    plt.imshow(red)
    plt.title("Red Channel")
    plt.show()

    plt.figure()
    plt.imshow(green)
    plt.title("Green Channel")
    plt.show()

    plt.figure()
    plt.imshow(blue)
    plt.title("Blue Channel")
    plt.show()        

    plt.figure()
    plt.imshow(green_mask)
    plt.title("Green Mask")
    plt.show()

    plt.figure()
    plt.imshow(redcyan_mask)
    plt.title("Red/Cyan Mask")
    plt.show()

    visualization = np.zeros((*straight_mask.shape, 3), dtype=np.uint8)
    visualization[straight_mask] = [255, 255, 255]  # White for straight edges
    visualization[non_straight_mask] = [0, 255, 0]  # Green for non-straight edges    

    plt.figure()
    plt.imshow(visualization)
    plt.title(f"Straight and Non-Straight Edges, SED: {sed:.4f}, NSED: {nsed:.4f}")
    plt.axis('off')
    plt.show()

    return result

do_for_each_jpeg_file_in_folder(folder_path, process_image)
